# glm-met

Example use of the glm-met Python package to download meteorological data and format it for use with the General Lake Model (GLM).

In [ ]:
# install glm-met - comment if this step is not required
!pip install glm-met

Here, we'll be working with the open-meteo Historical API which lets you download historical meteorological data based on the ERA-Land reanalysis product.

First, we need to create a `glm_met.openmeteo.historical.Historical` object, which stores API query attributes and has methods for downloading and processing the data.

In [3]:
import pprint
import pandas as pd
import os
import glm_met.openmeteo.historical as historical

variables = [
    "temperature_2m",
    "relativehumidity_2m",
    "precipitation",
    "cloudcover",
    "et0_fao_evapotranspiration",
    "windspeed_10m",
    "winddirection_10m",
    "windgusts_10m",
    "shortwave_radiation",
    "direct_normal_irradiance"
]

tmp_historical = historical.Historical(
        location=(116.69115540712589, -34.22581232495377),
        date_range=("2020-01-01", "2020-01-31"),
        variables=variables,
        met_data=None,
        glm_met_data=None,
        timezone="Australia/Sydney",
        hourly=True
    )

Once we have created the `Historical` object, `tmp_historical`, we can call its `get_variables()` method to make a request to the open-meteo Historical API and download the data. The downloaded data will be stored in `tmp_historical`'s `met_data` attribute. The `met_data` attribute's type is of `MetData` which has two attributes. `tmp_historical.met_data.metadata` is a dict of metadata about the request to the open-meteo Historical API and `tmp_historical.met_data.data` is a DataFrame object of meteorological data.

In [4]:
tmp_historical.get_variables(request_settings=None)

In [5]:
# view the metadata
pprint.pprint(tmp_historical.met_data.metadata)

{'elevation': 208.0,
 'generationtime_ms': 1.5289783477783203,
 'hourly_units': {'cloudcover': '%',
                  'direct_normal_irradiance': 'W/m²',
                  'et0_fao_evapotranspiration': 'mm',
                  'precipitation': 'mm',
                  'relativehumidity_2m': '%',
                  'shortwave_radiation': 'W/m²',
                  'temperature_2m': '°C',
                  'time': 'iso8601',
                  'winddirection_10m': '°',
                  'windgusts_10m': 'm/s',
                  'windspeed_10m': 'm/s'},
 'latitude': -34.2,
 'longitude': 116.70001,
 'timezone': 'Australia/Sydney',
 'timezone_abbreviation': 'AEST',
 'utc_offset_seconds': 36000}


In [6]:
# view the data
tmp_historical.met_data.data.head()

,time,temperature_2m,relativehumidity_2m,precipitation,cloudcover,et0_fao_evapotranspiration,windspeed_10m,winddirection_10m,windgusts_10m,shortwave_radiation,direct_normal_irradiance
0,2020-01-01T00:00,17.1,69,0.0,27,0.03,2.86,205,6.2,0.0,0.0
1,2020-01-01T01:00,16.2,75,0.0,22,0.01,1.94,192,5.3,0.0,0.0
2,2020-01-01T02:00,15.4,80,0.0,5,0.00,1.33,167,3.7,0.0,0.0
3,2020-01-01T03:00,14.6,85,0.0,0,0.00,1.30,113,2.6,0.0,0.0
4,2020-01-01T04:00,14.1,88,0.0,0,0.00,1.08,112,2.0,0.0,0.0


The GLM model requires meteorological data with a specific format and units. If the required meteorological variables to run GLM have been requested from open-meteo, `tmp_historical`'s `convert_to_glm()` method can be used reformat the data. The GLM formatted data is a DataFrame that is stored in `tmp_historical`'s `glm_met_data` attribute.

In [7]:
# convert data to GLM format
tmp_historical.convert_to_glm()

# visualise GLM formatted data
tmp_historical.glm_met_data.head()

,time,ShortWave,Cloud,AirTemp,RelHum,WindSpeed,Rain
0,2020-01-01 00:00,0.0,0.27,17.1,69,2.86,0.0
1,2020-01-01 01:00,0.0,0.22,16.2,75,1.94,0.0
2,2020-01-01 02:00,0.0,0.05,15.4,80,1.33,0.0
3,2020-01-01 03:00,0.0,0.00,14.6,85,1.30,0.0
4,2020-01-01 04:00,0.0,0.00,14.1,88,1.08,0.0


Finally, we can save the GLM formatted meteorological data to a csv file for use with the GLM model.

In [8]:
tmp_historical.write_glm_met(path=os.getcwd(), zip_f=False, fname="met.csv")

if "met.csv" in os.listdir(os.getcwd()):
    print("met.csv downloaded OK")

met.csv downloaded OK
